In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
import shutil
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.losses import BinaryCrossentropy

np.random.seed(1337)

In [2]:
import h5py

train_file = 'MA0035_4_m5_train.h5'
test_file = 'MA0035_4_m5_test.h5'

h5_train = h5py.File(train_file, 'r')
h5_test = h5py.File(test_file, 'r')

train_data = h5_train['data'][:90000]
train_binlabels = h5_train['binlabels'][:90000]

val_data = h5_train['data'][-10000:]
val_binlabels = h5_train['binlabels'][-10000:]

test_data = h5_test['data'][:]
test_binlabels = h5_test['binlabels'][:]

train_data = tf.cast(train_data, dtype=tf.float32)
train_labels = tf.cast(train_binlabels, dtype=tf.float32)

In [3]:
print(train_data.shape)
train_data_T = np.transpose(train_data, axes=(0, 2, 1))
print(train_data_T.shape)

(90000, 4, 1000)
(90000, 1000, 4)


In [4]:
print(test_data.shape)
test_data_T = np.transpose(test_data, axes=(0, 2, 1))
print(test_data_T.shape)

(10000, 4, 1000)
(10000, 1000, 4)


In [5]:
print(val_data.shape)
val_data_T = np.transpose(val_data, axes=(0, 2, 1))
print(val_data_T.shape)

(10000, 4, 1000)
(10000, 1000, 4)


In [6]:
def train_model(hyperparameters, log_dir):
    board_log = log_dir + '/board'
    hparams_log = log_dir + '/hparams.json'

    my_callbacks = [
        TensorBoard(log_dir=board_log),
        EarlyStopping('val_loss', patience=100)
    ]

    with open(hparams_log, 'w') as hparam_file:
        json.dump(hyperparameters, hparam_file)

    # model = tf.keras.models.Sequential(
    #     [
    #         tf.keras.layers.Flatten(input_shape=(4, 1000)),
    #         tf.keras.layers.Dropout(hyperparameters['l0_dropout_rate']),
    #         tf.keras.layers.Dense(hyperparameters['l1_hidden_units'], activation=tf.keras.activations.relu),
    #         # tf.keras.layers.Dense(hyperparameters['l2_hidden_units'], activation=tf.keras.activations.relu),
    #         # tf.keras.layers.Dropout(hyperparameters['l2_dropout_rate']),
    #         tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
    #     ]
    # )
    model = tf.keras.models.Sequential(
        [
            Flatten(input_shape=(1000, 4)),
            Dropout(hyperparameters['l0_dropout_rate']),
            Dense(100, activation='relu', kernel_regularizer='l2', bias_regularizer='l2'),
            Dropout(0.5),
            Dense(30, activation='relu'),
            Dropout(0.6),
            Dense(30, activation='relu', kernel_regularizer='l2', bias_regularizer='l2'),
            Dense(5, activation='relu'),
            Dense(1, activation=tf.keras.activations.sigmoid)
        ]
    )

    with tf.device('/GPU:0'):
        model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters['learning_rate']),
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.AUC(name='auc')],
        )

        history = model.fit(train_data_T, train_binlabels,
                epochs=hyperparameters['epochs'], 
                validation_data=(val_data_T, val_binlabels),
                batch_size=hyperparameters['batch_size'],
                callbacks=my_callbacks,
                shuffle=True)

    return model, history

In [7]:
def plot_and_save_roc(false_positive_rate, true_positive_rate, log_dir):
    roc = plt.figure(0)

    plt.plot(false_positive_rate, true_positive_rate, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")

    plt.savefig(log_dir + '/roc.pdf')
    pickle.dump(roc, open((log_dir + '/pickle/roc.fig.pickle'), 'wb'))

In [8]:
def plot_and_save_loss(history, log_dir):
    loss = plt.figure(1)
    history_loss = history.history['loss']
    history_val_loss = history.history['val_loss']
    epochs = range(len(history_loss))

    plt.plot(epochs, history_loss, 'ko', label='Training loss')
    plt.plot(epochs, history_val_loss, 'b', label='Validation loss')
    plt.ylim([0.0, 1.5])
    plt.title('Training and validation loss')
    plt.legend()

    plt.savefig(log_dir + '/loss.pdf')
    pickle.dump(loss, open((log_dir + '/pickle/loss.fig.pickle'), 'wb'))

In [9]:
def plot_and_save_auc(history, log_dir):
    auc = plt.figure(2)

    history_auc = history.history['auc']
    history_val_auc = history.history['val_auc']
    epochs = range(len(history_auc))

    plt.plot(epochs, history_auc, 'ko', label='Training auc')
    plt.plot(epochs, history_val_auc, 'b', label='Validation auc')
    plt.title('Training and validation auc')
    plt.ylim(0.46, 1.0)
    plt.legend()

    plt.savefig(log_dir + '/auc.pdf')
    pickle.dump(auc, open((log_dir + '/pickle/auc.fig.pickle'), 'wb'))

In [10]:
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 

In [11]:
L0_DROPOUT_RATES = [0.0, 0.5, 0.52, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68]
# L0_DROPOUT_RATES = [0.7]
# L1_DROPOUT_RATES = [0.0]
# L2_DROPOUT_RATES = [0.0]
# L1_HIDDEN_UNITS = [16]
# L2_HIDDEN_UNITS = [128]
BATCH_SIZE = [32, 64, 128, 254, 512]
# LEARNING_RATE = [0.000009]
LEARNING_RATE = [0.0000023]

# for l1_hidden_units in L1_HIDDEN_UNITS:
#     for learning_rate in LEARNING_RATE:
for l0_dropout_rate in L0_DROPOUT_RATES:
#             for l1_dropout_rate in L1_DROPOUT_RATES:
    log_dir = './logs/4_h_layer_test/:' + str(l0_dropout_rate) + '|hu100l2|dr0.5|hu30|dr0.6|hu30l2|hu5lr' + str(LEARNING_RATE[0])

    try:
        shutil.rmtree(log_dir)
    except OSError as e:
        print(e)

    try:
        os.makedirs(log_dir)
        os.makedirs(log_dir + '/pickle')
    except OSError as e:
        print(e)

    hparams = {
        'l0_dropout_rate': l0_dropout_rate,
        'learning_rate': LEARNING_RATE[0],
        'batch_size': BATCH_SIZE[2],
        'epochs': 300,
    }

    model, history = train_model(hparams, log_dir)

    acc, auc = model.evaluate(test_data_T, test_binlabels)
    yhat = model.predict(test_data)
    fpr, tpr, _ = roc_curve(test_binlabels, yhat)
    roc_auc = roc_auc_score(test_binlabels, yhat)

    plot_and_save_roc(fpr, tpr, log_dir)
    plot_and_save_loss(history, log_dir)
    plot_and_save_auc(history, log_dir)

    plt.show()
    model.save(log_dir + '/model')

Epoch 1/300
704/704 [==============================] - 3s 3ms/step - loss: 2.8957 - auc: 0.4992 - val_loss: 2.8548 - val_auc: 0.5050
Epoch 2/300
704/704 [==============================] - 2s 2ms/step - loss: 2.8308 - auc: 0.4962 - val_loss: 2.7948 - val_auc: 0.5024
Epoch 3/300
704/704 [==============================] - 2s 3ms/step - loss: 2.7670 - auc: 0.5041 - val_loss: 2.7353 - val_auc: 0.5024
Epoch 4/300
704/704 [==============================] - 2s 2ms/step - loss: 2.7099 - auc: 0.5006 - val_loss: 2.6765 - val_auc: 0.5008
Epoch 5/300
704/704 [==============================] - 2s 2ms/step - loss: 2.6492 - auc: 0.5034 - val_loss: 2.6172 - val_auc: 0.4994
Epoch 6/300
704/704 [==============================] - 2s 3ms/step - loss: 2.5893 - auc: 0.5027 - val_loss: 2.5572 - val_auc: 0.4976
Epoch 7/300
704/704 [==============================] - 2s 3ms/step - loss: 2.5274 - auc: 0.4986 - val_loss: 2.4980 - val_auc: 0.4984
Epoch 8/300
704/704 [==============================] - 2s 3ms/step - 